In [16]:
import copy
import random as rd
import csv
import math

In [17]:
nodeType = {
    "Entry":{
        "Usable Space":1,
        "Service Time high":1,
        "Service Time low":1,
        "Optional":False,
        "ChanceOfVisit":1
    },
    #     "Type 0":{
    #     "Usable Space":1,
    #     "Service Time high":240,
    #     "Service Time low":180,
    #     "Optional":True,
    #     "ChanceOfVisit":0.9
    # },
    "Type 1":{
        "Usable Space":1,
        "Service Time high":150,
        "Service Time low":90,
        "Optional":True,
        "ChanceOfVisit":0.9
    },
    "Type 2":{
        "Usable Space":1,
        "Service Time high":360,
        "Service Time low":240,
        "Optional":False,
        "ChanceOfVisit":1
    },
    "Type 3":{
        "Usable Space":1,
        "Service Time high":780,
        "Service Time low":660,
        "Optional":False,
        "ChanceOfVisit":1
    },
}

path1 = {
'source':'Entry',
'IntroSpaceL1':'Type 2',
'A':'Type 1',
'B':'Type 1',
'C':'Type 1',
'D':'Type 1',
'E':'Type 1',
'F':'Type 1',
'G':'Type 1',
'H':'Type 1',
'I':'Type 1',
'J':'Type 1',
'K':'Type 1',
'L':'Type 1',
'IntroSpaceL2':'Type 2',
'Area2':'Type 2',
'Area3':'Type 2',
'Area4':'Type 2',
'Area5':'Type 3',
}

spacesSkip = ['A','B','C','D','E','F','G','H','I','J','K','L']
# path4 = {'node24':90,'node25':30,'node26':60,'node27':60,'node28':40,'node29':20,'node30':60,'end':999}


# #Fast Track
# path10 = {'node49':90,'node27':60,'node3':60,'node57':30,'node64':30,'node36':60,'node12':60,'node51':60,'node37':40,'node13':40,
#           'node66':60,'node46':20,'node22':20,'node53':40,'node61':20,'node69':60,'end':999}

In [18]:
nodePaths = {'path1':path1,
            #    'path4':path4,
              } #nodename : service tieme in seconds
nodeRoundTime = {
    "path1": 180, #High
    "path2": 180, #High
    "path3": 180, #High
    "path4": 180, #Low
    "path5": 180,  #Low
    "path6": 180, #Low
}
pathLiftCapacity = {
    'path1': 81, #High
    'path2': 81, #High
    'path3': 81, #High
    'path4': 9, #Low
    'path5': 9,  #Low
    'path6': 9, #Low
}
# nodeDwellRange = 10 #seconds eg. 40 second dwell time with 10 second range, people can stay between 30-50 seconds

#For Lifts
liftRoundTime = 45 #seconds
liftCapacity = 90 #people

liftCapacityArrivalRate = {
    0:1,
    3600:1
}


In [19]:
def round_down(num, divisor):
    return num - (num%divisor)

In [20]:
class person:
    def __init__(self,personIndex,timeAtNode,pathList,nodeDetail):
        self.idx = personIndex
        self.curNode = pathList[0]
        self.curNodeIdx = 0
        self.timeAtCurNode = timeAtNode
        self.nodeList = pathList
        self.node_detail = nodeDetail 
        self.timeAtNextNode = timeAtNode + self.calculateTimeToNextNode()

    def __str__(self):
        return f"{self.idx} {self.curNode} {self.timeAtCurNode} {self.timeAtNextNode}"

    def calculateTimeToNextNode(self):
        # print(self.node_detail)
        lowRange = nodeType[self.node_detail[self.curNode]]['Service Time low']
        highRange = nodeType[self.node_detail[self.curNode]]['Service Time high']
        timeToSpendAtCurNode = rd.randrange(lowRange,highRange+1,1)
        return timeToSpendAtCurNode

    def updateAgentLocation(self,curTime):
        if curTime == self.timeAtNextNode:
            #Reached Last Step, remove details
            if self.curNode == self.nodeList[-1]:
                self.curNode = 'end'
                self.timeAtNextNode = -1
                return

            #if next node is optional, potentially skip it. But can't skip more than one at a time
            optional = nodeType[self.node_detail[self.nodeList[self.curNodeIdx+1]]]['Optional']
            if optional:
                chance = rd.random()
                if chance > nodeType[self.node_detail[self.nodeList[self.curNodeIdx+1]]]['ChanceOfVisit']:
                    #Skip this node
                    self.curNodeIdx += 2
                    self.curNode = self.nodeList[self.curNodeIdx]
                    self.timeAtCurNode = curTime
                    self.timeAtNextNode = curTime + self.calculateTimeToNextNode()
                    return
    
            self.curNodeIdx+=1
            self.curNode = self.nodeList[self.curNodeIdx]
            self.timeAtCurNode = curTime
            self.timeAtNextNode = curTime + self.calculateTimeToNextNode()
            return

In [21]:
#simulate arrival time of people coming in a lift and moving through from one to the next and calculate occupancy of each node over time
simulationTime = 18001 #seconds
#Escalators
peoplePerHour = 1500





#keep track of time when people arrive at a node, where they are and when they should leave.

arrivalRatePerSecond = peoplePerHour / 3600
numberOfPeopleGenerated = 0
numberOfPeople = 0
peopleList = []
nodeOccupancies = {}
for curTime in range(simulationTime):

    if curTime not in nodeOccupancies:
        nodeOccupancies[curTime] = {}
        for key in nodePaths:
            nodeOccupancies[curTime]['end'] = 0
            for paths in list(nodePaths[key]):
                nodeOccupancies[curTime][paths] = 0

    numberOfPeopleGenerated += arrivalRatePerSecond
    ##Constant Arrival
    personArrivalAmount = math.floor(numberOfPeopleGenerated)

    for idx,paths in enumerate(nodePaths):
        nodeDetails = nodePaths[paths]
        nodeList = list(nodeDetails)
############################################################################################################
        ## Lift Arrival Methodd
        # offset = idx * 12
        # if (curTime+offset)%nodeRoundTime[paths] == 0:# People Arrive at node 1
        #     timeBucket = round_down(curTime, 3600)
        #     liftOccupancy = int(pathLiftCapacity[paths] * liftCapacityArrivalRate[timeBucket])
        #     for personIndex in range(liftOccupancy):
        #         peopleList.append(person(len(peopleList),curTime,nodeList,nodeDetails))
##############################################################################################################        



        for personIndex in range(int(personArrivalAmount)):
            peopleList.append(person(len(peopleList),curTime,nodeList,nodeDetails))

    numberOfPeopleGenerated-=math.floor(numberOfPeopleGenerated)

    for agent in peopleList:
        agent.updateAgentLocation(curTime)
        agentLocation = agent.curNode
        nodeOccupancies[curTime][agentLocation] += 1



In [22]:
# Get headers from one of the inner dictionaries
fieldnames = ['time'] + list(next(iter(nodeOccupancies.values())).keys())

with open('RyiadhPavilion_StaticCalcs_RandomSkip.csv', 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for time, node_values in nodeOccupancies.items():
        row = {'time': time}
        row.update(node_values)
        writer.writerow(row)